In [44]:
import pandas as pd
import numpy as np


In [45]:
test_ad=pd.read_csv("..\\test\\ad.csv")
test_click_log=pd.read_csv("..\\test\\click_log.csv")

In [46]:
train_ad=pd.read_csv("..\\train_preliminary\\ad.csv")
train_click_log=pd.read_csv("..\\train_preliminary\\click_log.csv")
train_user=pd.read_csv("..\\train_preliminary\\user.csv")

In [47]:
train_ad

,creative_id,ad_id,product_id,product_category,advertiser_id,industry
0,1,1,\N,5,381,78
1,4,4,\N,5,108,202
2,7,7,\N,5,148,297
3,8,8,\N,5,713,213
4,9,9,\N,5,695,213
...,...,...,...,...,...,...
2481130,4445714,3812196,9152,3,56976,248
2481131,4445715,3812197,39063,17,427,\N
2481132,4445716,3812198,39063,17,427,\N
2481133,4445717,3812199,35885,17,427,\N


In [48]:
train_click_log

,time,user_id,creative_id,click_times
0,9,30920,567330,1
1,65,30920,3072255,1
2,56,30920,2361327,1
3,6,309204,325532,1
4,59,309204,2746730,1
...,...,...,...,...
30082766,6,30920,228129,1
30082767,57,30920,2361397,1
30082768,26,30920,629802,1
30082769,86,30920,2713031,1


In [49]:
train_user

,user_id,age,gender
0,1,4,1
1,2,10,1
2,3,7,2
3,4,5,1
4,5,4,1
...,...,...,...
899995,899996,5,1
899996,899997,3,2
899997,899998,4,2
899998,899999,3,1


In [51]:
train_merged=pd.merge(train_ad,train_click_log,on='creative_id')
train_merged=pd.merge(train_merged,train_user,on='user_id')
display(train_merged)

MemoryError: Unable to allocate array with shape (2, 30082771) and data type int64

In [43]:
train_merged[train_merged["user_id"]==30920]

,time,user_id,creative_id,click_times
0,9,30920,567330,1
1,65,30920,3072255,1
2,56,30920,2361327,1
30082732,81,30920,3879497,1
30082733,80,30920,3751623,1
30082734,17,30920,188898,1
30082735,46,30920,2164075,1
30082736,49,30920,2085566,1
30082737,64,30920,2901618,1
30082738,37,30920,851785,1
